# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("output_data/cities.csv")
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
0,0,bilibino,68.05,166.44,26.31,26.31,95,69,4.12
1,1,dhrangadhra,22.98,71.47,87.30,87.30,37,0,11.41
2,2,cape town,-33.93,18.42,64.36,68.00,67,96,8.05
3,3,albany,42.60,-73.97,59.29,60.01,26,75,4.70
4,4,kanniyakumari,8.08,77.57,84.61,84.61,77,100,10.42


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
coord = weather_df[['Latitude', 'Longitude']]

In [4]:
figure_layout = {'width' : '800px', 'height' : '600px', 'border' : '1px solid black', 'padding' : '1px',
                 'margin' : '0 auto 0 auto'}
fig = gmaps.figure(layout = figure_layout)
humidity = gmaps.heatmap_layer(coord, weights = weather_df['Humidity (%)'], dissipating = False, max_intensity = 100, point_radius = 5)
fig.add_layer(humidity)
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_cities_df = weather_df.loc[(weather_df['Max Temperature (F)'] > 70) & (weather_df['Max Temperature (F)'] < 80) &
                                    (weather_df['Wind Speed (MPH)'] < 10) & (weather_df['Cloudiness (%)'] == 0)]
vacation_cities_df

,Unnamed: 0,City,Latitude,Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
34,34,nosy varika,-20.58,48.53,75.06,75.06,85,0,8.99
70,70,gobabis,-22.45,18.97,78.80,78.80,16,0,2.24
73,73,namibe,-15.20,12.15,79.20,79.20,53,0,9.31
96,96,viransehir,37.24,39.76,76.37,76.37,41,0,6.51
295,295,ravar,31.27,56.81,75.67,75.67,16,0,5.86
366,366,wanning,18.80,110.40,77.29,77.29,82,0,3.11
448,448,hwange,-18.37,26.50,73.80,73.80,21,0,3.74
558,558,taltal,-25.40,-70.48,70.34,70.34,45,0,5.48


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

In [7]:
hotels = []
countries = []
for lat, lon in zip(vacation_cities_df['Latitude'], vacation_cities_df['Longitude']):
    
    query_url = f'{base_url}location={lat},{lon}&radius=5000&type=hotel&key={g_key}'
    response = requests.get(query_url).json()
    
    try:
        hotels.append(response['results'][1]['name'])
        countries.append(response['results'][1]['plus_code']['compound_code'])
    except TypeError:
        print('failed')
        
countries

['CG8M+M2 Nosy-Varika, Madagascar',
 'HX24+H9 Gobabis, Namibia',
 'Q5R5+3G Namibe, Angola',
 '6QM7+4G Viranşehir, Şanlıurfa, Turkey',
 '7R84+CP Ravar, Kerman Province, Iran',
 'Q9RC+C7 Wanning, Hainan, China',
 'JFMR+CV Hwange, Zimbabwe',
 'HGR8+C7 Taltal, Chile']

In [8]:
vaca_countries = []
for countries in countries:
    split_countries = countries.split(',')
    vaca_countries.append(split_countries[-1])
vaca_countries


[' Madagascar',
 ' Namibia',
 ' Angola',
 ' Turkey',
 ' Iran',
 ' China',
 ' Zimbabwe',
 ' Chile']

In [13]:
vacation_cities_df['Hotel Name'] = hotels
vacation_cities_df['Country'] = vaca_countries
vacation_cities_df

C:\Users\flury\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\flury\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,City,Latitude,Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Hotel Name,Country
34,34,nosy varika,-20.58,48.53,75.06,75.06,85,0,8.99,Hotel Volazara,Madagascar
70,70,gobabis,-22.45,18.97,78.80,78.80,16,0,2.24,Goba Lodge,Namibia
73,73,namibe,-15.20,12.15,79.20,79.20,53,0,9.31,Pavilhão Welwitschia Mirabilis,Angola
96,96,viransehir,37.24,39.76,76.37,76.37,41,0,6.51,Bellona-ışık Ltd. Şti.,Turkey
295,295,ravar,31.27,56.81,75.67,75.67,16,0,5.86,Tejarat Bank,Iran
366,366,wanning,18.80,110.40,77.29,77.29,82,0,3.11,Wanning No.3 Middle School,China
448,448,hwange,-18.37,26.50,73.80,73.80,21,0,3.74,Fred Driver & Son (PVT) Ltd,Zimbabwe
558,558,taltal,-25.40,-70.48,70.34,70.34,45,0,5.48,Residencial Karime,Chile


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_cities_df.iterrows()]
locations = vacation_cities_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations,  fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2, info_box_content =  hotel_info)
fig.add_layer(hotel_layer)
fig

# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…